In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset, random_split
from PIL import Image

# Hiperparametreler
batch_size = 32
learning_rate = 0.0001
num_epochs = 5
test_split_ratio = 0.2

# Özel Dataset sınıfı
class SignatureDataset(Dataset):
    def __init__(self, genuine_dir, forgery_dir, transform=None):
        self.genuine_dir = genuine_dir
        self.forgery_dir = forgery_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        
        # Genuine klasöründen gerçek imzaları ekle
        for person_id in os.listdir(genuine_dir):
            person_path = os.path.join(genuine_dir, person_id)
            if os.path.isdir(person_path):
                for img_name in sorted(os.listdir(person_path)):
                    img_path = os.path.join(person_path, img_name)
                    self.image_paths.append(img_path)
                    self.labels.append(0)  # Gerçek imza için 0

        # Forgery klasöründen sahte imzaları ekle
        for forgery_type in os.listdir(forgery_dir):
            forgery_type_path = os.path.join(forgery_dir, forgery_type)
            if os.path.isdir(forgery_type_path):
                for person_id in os.listdir(forgery_type_path):
                    person_path = os.path.join(forgery_type_path, person_id)
                    if os.path.isdir(person_path):
                        for img_name in sorted(os.listdir(person_path)):
                            img_path = os.path.join(person_path, img_name)
                            self.image_paths.append(img_path)
                            self.labels.append(1)  # Sahte imza için 1

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

# Veri dönüşümleri
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Dataset oluşturma
full_dataset = SignatureDataset(genuine_dir="1_veri_setleri/2_UTSig/Genuine", forgery_dir="1_veri_setleri/2_UTSig/Forgery/Skilled", transform=data_transforms)

# Eğitim ve test setlerine bölme
test_size = int(len(full_dataset) * test_split_ratio)
train_size = len(full_dataset) - test_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# DataLoader oluşturma
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Modeli yükleme ve ayarlama
model = models.vgg16(pretrained=True)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, 2)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss ve optimizer tanımlama
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Eğitim fonksiyonu
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        accuracy = correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}")

# Test fonksiyonu
def test_model(model, test_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    test_loss = running_loss / len(test_loader.dataset)
    accuracy = correct / total
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {accuracy:.4f}")

# Modeli eğit
train_model(model, train_loader, criterion, optimizer, num_epochs)

# Modeli test et
test_model(model, test_loader, criterion)


Epoch [1/5], Loss: 0.0109, Accuracy: 0.9928
Epoch [2/5], Loss: 0.0000, Accuracy: 1.0000
Epoch [3/5], Loss: 0.0000, Accuracy: 1.0000
Epoch [4/5], Loss: 0.0000, Accuracy: 1.0000


UnidentifiedImageError: cannot identify image file '/Users/hbasriuzun/Documents/GitHub/Signature_recegnator/1_models/1_veri_setleri/2_UTSig/Genuine/106/11.tif'